## Terminology

- **Trial**: A single occurrence with an outcome that is uncertain until we observe it.
For example, rolling a single die.
- **Outcome**: A possible result of a trial; one particular state of the world. What Laplace calls a case.
For example: 4.
- **Sample Space**: The set of all possible outcomes for the trial.
For example, $\{1, 2, 3, 4, 5, 6\}$.
- **Event**: A subset of the sample space, a set of outcomes that together have some property we are interested in.
For example, the event "odd die roll" is the set of outcomes $\{1,3,5\}$.
- **Probability**: As Laplace said, the probability of an event with respect to
  a sample space is the "number of favorable cases" (outcomes from the sample
  space that are in the event) divided by the "number of all the cases" in the
  sample space (assuming "nothing leads us to expect that any one of these cases
  should occur more than any other"), so $P \in [0,1]$.
  For example, the probability of an odd die roll is $3/6 = 1/2$.


## Introduction

### Dice rolling implementation

In [20]:
from fractions import Fraction

def P(event, space):
    "The probability of an event, given a sample space."
    return Fraction(cases(favorable(event, space)),cases(space))

favorable = set.intersection # Outcomes that are in the event and in the sample space
cases     = len              # The number of cases is the length, or size, of a set

D     = {1, 2, 3, 4, 5, 6} # a sample space
even  = {   2,    4,    6} # an event
# NB: intersected with sample space to get favorable outcomes
odd   = {1, 3, 5, 7, 9, 11, 13}
prime = {2, 3, 5, 7, 11, 13}

In [21]:
P(even, D)

Fraction(1, 2)

In [22]:
P(odd, D)

Fraction(1, 2)

In [23]:
P((even | prime), D) # The probability of an even or prime die roll

Fraction(5, 6)

In [24]:
P((odd & prime), D) # The probability of an odd prime die roll

Fraction(1, 3)

### Card problems

In [25]:
import itertools
import random

def combos(items, n):
    "All combinations of n items; each combo as a space-separated str."
    return list(set(map(' '.join, itertools.combinations(items, n))))

suits = u'♥♠♦♣'
ranks = u'AKQJT98765432'
deck  = [r + s for r in ranks for s in suits] # 52
# sample space of all 5-card combinations from deck
Hands = combos(deck, 5) # 2598960

In [26]:
random.sample(Hands, 7)


['J♦ 9♦ 8♣ 4♠ 3♥',
 'J♠ 8♦ 6♣ 5♣ 3♠',
 'Q♥ Q♠ 8♥ 7♠ 3♠',
 'A♠ J♣ 9♥ 7♦ 2♣',
 'Q♣ 6♠ 4♦ 3♥ 3♣',
 'J♠ J♣ T♥ 7♥ 3♦',
 'A♣ Q♥ 9♠ 9♦ 7♠']

In [27]:
random.sample(deck, 7)

['T♥', '9♠', '3♣', '3♦', '2♣', 'K♦', 'J♦']

In [28]:
# 5 cards of the same suit
flush = {hand for hand in Hands if any(hand.count(suit) == 5 for suit in suits)}
P(flush, Hands)

Fraction(33, 16660)

In [29]:
four_kind = {hand for hand in Hands if any(hand.count(rank) == 4 for rank in ranks)}
P(four_kind, Hands) # e.g. 4 kings

Fraction(1, 4165)

## Urn Problems

> *An urn contains 6 blue, 9 red, and 8 white balls.  We select six balls at random. What is the probability of each of these  outcomes:*
> 
> - *All balls are red*.
> - *3 are blue, and 1 is red, and 2 are white, *.
> - *Exactly 4 balls are white*.

In [34]:
def balls(color, n):
    "A set of n numbered balls of the given color."
    return {color + str(i)
            for i in range(1, n + 1)}

urn = list(balls('B', 6) | balls('R', 9) | balls('W', 8))
random.sample(urn, 4)

['B4', 'R4', 'R3', 'R1']


## References

- Peter Norvig's [ipynb](https://github.com/norvig/pytudes/blob/main/ipynb)
-